# AAAAAAAAA

In [ ]:
# /// script
# requires-python = ">=3.11"
# dependencies = [
#     "pandas",
#     "quak"
# ]
# ///

In [ ]:
%load_ext quak
%load_ext autoreload
%autoreload 2

In [ ]:
from civicdbToBed import *

In [ ]:
import datetime
day = datetime.date.today() 
day = datetime.date.fromisoformat("2024-10-01")
day = datetime.date.fromisoformat("2024-11-01")
get_date_slug(day)

In [ ]:
dfs = download_and_load_dataframes(DOWNLOAD_DIR, DOWNLOAD_BASE_URL, day, overwrite=False, skip_check_prior=True)

In [ ]:
dfs["VariantSummaries"]

In [ ]:
fdf = dfs["VariantSummaries"].query("not is_flagged and feature_type == 'Fusion'")
pd.crosstab(~ fdf["5_prime_transcript"].isnull(), ~ fdf["5_prime_end_exon"].isnull())

In [ ]:
fdf

In [ ]:
filtered_variant = filter_augment_variant_summaries(dfs["VariantSummaries"])
filtered_variant

In [ ]:
adf = transform_assertion_summaries(
    dfs["AssertionSummaries"], dfs["MolecularProfileSummaries"]
)
edf = transform_clinical_evidence(
    dfs["ClinicalEvidenceSummaries"], dfs["MolecularProfileSummaries"]
)
#vldf = transform_variant_summaries_to_loci(dfs["VariantSummaries"])

mps = dfs["MolecularProfileSummaries"]

mid2aids = split_one_to_n_map(mps, "molecular_profile_id", "assertion_ids")
mid2eids = split_one_to_n_map(mps, "molecular_profile_id", "evidence_item_ids")
mid2vids = split_one_to_n_map(mps, "molecular_profile_id", "variant_ids")

In [ ]:
edf

In [ ]:
dfs["ClinicalEvidenceSummaries"]

In [ ]:
dfs["AssertionSummaries"]

In [ ]:
from collections import defaultdict
vid2mids = defaultdict(list)
for mid, vids in mid2vids.items():
    for vid in vids:
        vid2mids[vid].append(mid)

In [ ]:
vid2diseases = defaultdict(set)
vid2therapies = defaultdict(set)
missing_mps = []
for _, mpid, disease, therapies in dfs["AssertionSummaries"][["molecular_profile_id", "disease", "therapies"]].itertuples():
    tset = set(therapies.split(",")) if therapies == therapies else set()
    dset = set([disease]) if disease == disease else set()
    
    if mpid in mid2vids:
        for vid in mid2vids[mpid]:
            vid2diseases[vid] |= dset
            vid2therapies[vid] |= tset
    else:
        missing_mps.append(mpid)


for _, mpid, disease, therapies in dfs["ClinicalEvidenceSummaries"][["molecular_profile_id", "disease", "therapies"]].itertuples():
    tset = set(therapies.split(",")) if therapies == therapies else set()
    dset = set([disease]) if disease == disease else set()
    
    if mpid in mid2vids:
        for vid in mid2vids[mpid]:
            vid2diseases[vid] |= dset
            vid2therapies[vid] |= tset
    else:
        missing_mps.append(mpid)

disease_df = pd.DataFrame({"diseases": [", ".join(x) for x in vid2diseases.values()]}, index=vid2diseases.keys())

therapy_df = pd.DataFrame({"therapies": [", ".join(x) for x in vid2therapies.values()]}, index=vid2therapies.keys())

dfs["VariantSummaries"].set_index("variant_id").join(disease_df).join(therapy_df).reset_index()

In [ ]:
vid2therapies[3]

In [ ]:
dfs["VariantSummaries"].loc[dfs["VariantSummaries"]["feature_type"] == "Factor", :]

In [ ]:
mps.loc[mps['molecular_profile_id'] == 4344, :]

In [ ]:
ces = dfs["ClinicalEvidenceSummaries"].set_index("evidence_id")

In [ ]:
ces.loc[mid2eids[12],]

In [ ]:
dfs["ClinicalEvidenceSummaries"]

# Disease ontology

DOID links look like https://www.disease-ontology.org/?id=DOID:1909

so the doid column needs to be converted to an integer and then prepended with the string https://www.disease-ontology.org/?id=DOID:

In [ ]:
dfs["AssertionSummaries"]

In [ ]:
#disease	doid
DOID_BASE_URL = "https://www.disease-ontology.org/?id=DOID:{}"
dfs = load_dataframes(dl_dir)
#dfs["AssertionSummaries"]["doid"] = dfs["AssertionSummaries"]["doid"].astype("Int64").astype("str")



In [ ]:
dfs["AssertionSummaries"]["doid"] == "<NA>"